Invoke basic libraries and give input file path directory

In [47]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

The Matplotlib package is invoked for visualisations and the sklearn library for regression steps to do train, testsplit and get model accuracy

In [48]:
import os
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

#from os import path
#os.chdir('/kaggle/input/significance measure of product search')

Invoking nltk for lemmetisation which weren't useful hence used pattern library later in below codes

In [49]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer # Root / Base Words.
import re # Regex.
import string
from sklearn.feature_extraction import text
ps = PorterStemmer()
lm = WordNetLemmatizer()

The Stopwords library is called for the stopwords removal

In [50]:
# conda install -c conda-forge wordcloud (one time install)
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from PIL import Image

Call the spacy, Counter functions for NLP tasks, word counting tasks

In [51]:
# !python -m spacy download en_core_web_sm (One time)
import spacy
from spacy import displacy 
from collections import Counter
import en_core_web_sm # en --> English pre-trained 
# sm --> Small Model || md --> Medium Model || lg --> Large Model
# Loading large model in python:  --> gensim, tokenize & then lemmi.
nlp = spacy.load('en_core_web_sm')

Invoke the datasets-train, attributes, product_descriptions, test in variables train, attr, desc, test respectively from the files uploaded in Input directory data 

In [52]:
train = pd.read_csv('../input/significance-measure-of-product-search/train.csv',encoding = 'ISO-8859-1')
attr = pd.read_csv('../input/significance-measure-of-product-search/attributes.csv')
desc = pd.read_csv('../input/significance-measure-of-product-search/product_descriptions.csv', encoding='ISO-8859-1')
test = pd.read_csv('../input/significance-measure-of-product-search/test.csv', encoding = 'ISO-8859-1')
#ISO 8859-1 encodes as "Latin alphabet no. 1", consisting of 191 characters from the Latin script.

Find the dimensions of the respective datasets using shape function

In [53]:
print(train.shape)
print(attr.shape)
print(desc.shape)
print(test.shape)

Find null values of train dataset

In [54]:
train.isnull().sum()

Give the bin range for relevance plot, and the occurences in the respective slabs

In [55]:
s_bin = [0.99,1,1.5,2,2.5,3]
group_bin = train.groupby(pd.cut(train['relevance'],s_bin))
group_bin.size()

Import seaborn package thereby to plot the relevance score distribution of train dataset

In [56]:
import seaborn as sns
sns.catplot(x='relevance',kind = 'count', data = train,height = 5, aspect = 2.5, order=[1,1.33,1.67,2,2.33,2.67,3])
plt.title('Relevance Scores Spread in Train Data')
plt.xlabel('Relevance Scores')
plt.show()

Find the null values in product description dataset

In [57]:
desc.isnull().sum()

Find the null values in attributes dataset

In [58]:
attr.isnull().sum()

Removing common null values in attributes dataset in all prod uid, name, values (155 values dropped)

In [59]:
# Removing common null values:
attr.loc[attr['product_uid'].isna()].head()

Display of the remaining null values present in value column of attributes dataset

In [60]:
attr = attr.dropna(subset = ['product_uid'])
attr.isnull().sum()

Drop the id columns from the train and test datasets as they represent just the serial number and not used in modelling.

In [61]:
# Dropping 'id':
train = train.drop('id', axis = 1)
test = test.drop('id', axis = 1)

Merging the product descriptions with train and test dataset using left join with common product uid

In [62]:
## Merging product_desc. with train:
train = pd.merge(train,desc, how = 'left', on = 'product_uid' )
print(train.shape)
train.head()

In [63]:
test = pd.merge(test,desc, how = 'left', on = 'product_uid')
print(test.shape)
test.head()

Find maximum occurence of specific types in name column of attributes dataset using value_counts() function

In [64]:
attr['name'].value_counts().head(7)

# Out of 1,24,428 unique IDS,MFG Brand Name is repeated most times (70%)

Display the MFG Brand name columns in the attributes dataset

In [65]:
attr.loc[(attr['name'] == 'MFG Brand Name')].head(10)

Separates brand column from attributes and assign in brand_attr variable

In [66]:
attr['brand'] = attr.loc[(attr['name'] == 'MFG Brand Name'), 'value']
brand_attr = attr.drop(['name','value'], axis=1)
brand_attr.isnull().sum()

Drop the null values in brand_attr

In [67]:
## Dropping null values:
brand_attr = brand_attr.dropna(axis = 'rows')
brand_attr.head()

Merge the brand_attr with train and test dataset through left join via common product_uid

In [68]:
## Merging brand with train:
train = pd.merge(train,brand_attr,how='left',on='product_uid')

In [69]:
## Merging brand with test:
test = pd.merge(test,brand_attr,how='left',on='product_uid')

Drop the brand column in the attributes dataset as it is merged with train, test respectively

In [70]:
## Dropping brand:
attr = attr.drop('brand', axis = 'columns')

Drop the MFG Brand name containing names column in attributes dataset, to merge effectively as already used.

In [71]:
# Dropping rows that contains 'MFG Brand Name':
attr.drop(attr.query("name == 'MFG Brand Name'").index, inplace=True)

Display the modified attributes dataset after modifications as stated in above steps

In [72]:
attr.head(10)

Concatenate the name and value columns of attributes dataset into combined_attr new column

In [73]:
# Concatening 2 text data:
attr['combined_attr'] = attr['name'].str.cat(attr['value'],sep=' ')
attr.head()

Print the dimensions and check presence of null values in attributes dataset

In [74]:
print(attr.shape)
attr.isnull().sum()

Locate and compare the null values in the value column with other columns 

In [75]:
attr.loc[attr['value'].isna()]

Dropping the null values in attributes dataset as name has no value specified

In [76]:
# Dropping null values since attribute name has no value
attr = attr.dropna()
attr.shape

Dropping name, value columns in attributes dataset as we are using only combined_attr for analysis 

In [77]:
attr = attr.drop(['name','value'], axis='columns')

Print the top 10 rows of the modified attributes dataset

In [78]:
attr.head(10)

Group the attributes so that combined_attr are got with unique respective product uIDS and merged using join row wise

In [79]:
# Grouping all attributes based on unique IDs:
attr = attr.groupby(['product_uid'])['combined_attr'].apply(' '.join).reset_index()
attr.head()
# Reset_index drops the index value thereby can perform grouping in same data!

Merging combines attributes column with train and test dataset via common product uid

In [80]:
# Merging Combined_attributes with train:
train_c = pd.merge(train,attr[['product_uid','combined_attr']],how='left',on='product_uid')

In [81]:
# Merging Combined_attributes with test:
test_c = pd.merge(test,attr[['product_uid','combined_attr']],how='left',on='product_uid')
test_c.head()

Find null values in modified test dataset 

In [82]:
test_c.isnull().sum()

Print the dimensions of train dataset and the top 10 rows

In [83]:
print(train_c.shape)
train_c.head(10)

Find the null values of train dataset through isnull()

In [84]:
train_c.isnull().sum()

Check the presence of non null values in brand column and compare side by side with the value_counts()

In [85]:
# Checking non-null values of brand & comparing it with product_title:
print(train_c.loc[~train_c['brand'].isna(),['product_title','brand']].value_counts().head(35))

Finding is that Brand contains atleast 1 word to atmost 4 words from product title column

In [86]:
## It can be noted that brand contains atleast 1 word to atmost 4 words from product title:
brand_u = np.unique(train_c['brand'].dropna())
print(brand_u.shape)
brand_u

Check unique brands available

In [87]:
## It can be noted that brand contains atleast 1 word to atmost 4 words from product title:
brandtest_u = np.unique(test_c['brand'].dropna())
print(brandtest_u.shape)
brandtest_u

Functions to replace null values in brand by 1st four words occuring in product title column of both train and test

In [88]:
def word(n,title):                            # n --> No. of words & joining them with spaces in product_title
    return ' '.join(title.split()[:n])        # Defining the merge in iteration!

In [89]:
def fillna_brand(dummy_tr, brand_u):              # Defining temp. data (for filling null) & unique brands:
    missing = dummy_tr[dummy_tr['brand'].isna()].copy()
    
    for i, row in missing.iterrows():         # Iterate each row in the DataFrame
            title = row['product_title']      # title --> All Rows of the product_title
            if word(4, title) in brand_u:
                    missing.loc[i,'brand'] = word(4,title)
            elif word(3, title) in brand_u:
                    missing.loc[i,'brand']  = word(3,title)
            elif word(2, title) in brand_u:
                    missing.loc[i,'brand']  = word(2,title)
            else:
                    missing.loc[i,'brand']  = word(1,title)
    dummy_tr.loc[missing.index,'brand'] = missing['brand']
    return dummy_tr

In [90]:
def fillna_brand(dummy_tr, brandtest_u):              # Defining temp. data (for filling null) & unique brands:
    missing = dummy_tr[dummy_tr['brand'].isna()].copy()
    
    for i, row in missing.iterrows():         # Iterate each row in the DataFrame
            title = row['product_title']      # title --> All Rows of the product_title
            if word(4, title) in brandtest_u:
                    missing.loc[i,'brand'] = word(4,title)
            elif word(3, title) in brandtest_u:
                    missing.loc[i,'brand']  = word(3,title)
            elif word(2, title) in brandtest_u:
                    missing.loc[i,'brand']  = word(2,title)
            else:
                    missing.loc[i,'brand']  = word(1,title)
    dummy_tr.loc[missing.index,'brand'] = missing['brand']
    return dummy_tr

In [91]:
train_c = fillna_brand(train_c,brand_u)
train_c.isnull().sum()

In [ ]:
test_c = fillna_brand(test_c,brandtest_u)
test_c.isnull().sum()

In [ ]:
train_c['brand'].value_counts().head(10)

In [ ]:
## Intrepretation non-null values in attributes: (Similar to description)
train_c.loc[(~train_c['combined_attr'].isna(),['combined_attr','product_description'])]

Replace null values in combined attr with prod_desc since both are similar

In [ ]:
# Viewing null values for verification:
train_c.loc[(train_c['combined_attr'].isna(),['combined_attr','product_description'])]

In [ ]:
# Filling null values in combined_attributes:
train_c['combined_attr'] = train_c['combined_attr'].fillna(train_c.loc[train_c['combined_attr'].isna(),'product_description'])
train_c.isnull().sum()

In [ ]:
# Filling null values in combined_attributes of TEST:
test_c['combined_attr'] = test_c['combined_attr'].fillna(test_c.loc[test_c['combined_attr'].isna(),'product_description'])
test_c.isnull().sum()

In [ ]:
print(train_c.iloc[302])
print(train_c.iloc[303]) # Checking the consolidated file

In [ ]:
train_c.head(20)

Concatenating train and test_data into a single dataset named features

In [ ]:
#Concatenating train_)data and test_data into dtaframe features
features = pd.concat((train_c, test_c), axis=0, ignore_index=True)
print(str(len(train_c)) + ' ' + str(len(test_c)) + ' ' + str(len(features)))
#del train_le
#del test

TEXT MINING steps (string_edit function applied) - 
1)Text treatment (lower)
2)Units standardisation
3)Special characters removal

In [ ]:
def string_edit(s:str): 
    if isinstance(s, str):
        s = re.sub(r"(\w)\.([A-Z])", r"\1 \2", s) 
        s = s.lower()
        s = s.replace("  "," ")
        s = s.replace(",","") 
        s = s.replace("$"," ")
        s = s.replace("?"," ")
        s = s.replace("-"," ")
        s = s.replace("//","/")
        s = s.replace("..",".")
        #s = s.replace(" / "," ") #it will convert fractional nu. to whole nu.
        s = s.replace(" \\ "," ")
        s = s.replace("."," . ")
        #s = re.sub(r"(^\.|/)", r"", s)
        s = re.sub(r"(\.|/)$", r"", s)
        s = re.sub(r"([0-9])([a-z])", r"\1 \2", s)
        s = re.sub(r"([a-z])([0-9])", r"\1 \2", s)
        #s = s.replace(" x "," xbi ")
        s = re.sub(r"([a-z])( *)\.( *)([a-z])", r"\1 \4", s)
        s = re.sub(r"([a-z])( *)/( *)([a-z])", r"\1 \4", s)
        s = s.replace("*"," x ")
        s = s.replace(" by "," x ") #search term (100047)
        s = re.sub(r"([0-9])( *)\.( *)([0-9])", r"\1.\4", s)
        
        # Consolidate variations of equivalent unit terms 
        s = re.sub(r"([0-9]+)( *)(inches|inch|in|')\.?", r"\1in. ", s)
        s = re.sub(r"([0-9]+)( *)(foot|feet|ft|'')\.?", r"\1ft. ", s)
        s = re.sub(r"([0-9]+)( *)(pounds|pound|lbs|lb)\.?", r"\1lb. ", s)
        s = re.sub(r"([0-9]+)( *)(square|sq) ?\.?(feet|foot|ft)\.?", r"\1sq.ft. ", s)
        s = re.sub(r"([0-9]+)( *)(cubic|cu) ?\.?(feet|foot|ft)\.?", r"\1cu.ft. ", s)
        s = re.sub(r"([0-9]+)( *)(gallons|gallon|gal)\.?", r"\1gal. ", s)
        s = re.sub(r"([0-9]+)( *)(ounces|ounce|oz)\.?", r"\1oz. ", s)
        s = re.sub(r"([0-9]+)( *)(centimeters|cm)\.?", r"\1cm. ", s)
        s = re.sub(r"([0-9]+)( *)(milimeters|mm)\.?", r"\1mm. ", s)
        s = s.replace("°"," degrees ")
        s = re.sub(r"([0-9]+)( *)(degrees|degree)\.?", r"\1deg. ", s)
        s = s.replace(" v "," volts ")
        s = re.sub(r"([0-9]+)( *)(volt\|volt)\.?", r"\1volt. ", s)
        s = re.sub(r"([0-9]+)( *)(watts|watt)\.?", r"\1watt. ", s)
        s = re.sub(r"([0-9]+)( *)(amperes|ampere|amps|amp)\.?", r"\1amp. ", s)
        s = s.replace("  "," ")
        s = s.replace(" . "," ")
        
        # Handling numeric instances with common identifiers
        s = re.sub(r"zero\.?", r"0 ", s)
        s = re.sub(r"one\.?", r"1 ", s)
        s = re.sub(r"two\.?", r"2 ", s)
        s = re.sub(r"three\.?", r"3 ", s)
        s = re.sub(r"four\.?", r"4 ", s)
        s = re.sub(r"five\.?", r"5 ", s)
        s = re.sub(r"six\.?", r"6 ", s)
        s = re.sub(r"seven\.?", r"7 ", s)
        s = re.sub(r"eight\.?", r"8 ", s)
        s = re.sub(r"nine\.?", r"9 ", s)
        
        return s
    else:
        # Return a "null" string if the parameter supplied is not a string
        return "null"

In [ ]:
features['search_term'] = features['search_term'].apply(str).apply(string_edit)
features['product_title'] = features['product_title'].apply(str).apply(string_edit)
features['combined_attr'] = features['combined_attr'].apply(str).apply(string_edit)
features['brand'] = features['brand'].apply(str).apply(string_edit)
features['product_description'] = features['product_description'].apply(str).apply(string_edit)

4. HTML Tag removal (BeautifulSoup, lxml)

In [ ]:
print('Total {} html tags contains in product description'.format(features.product_description.str.count('<br$').values.sum()))
print('Total {} html tags contains in combined_attr'.format(features.combined_attr.str.count('<br$').values.sum()))
print('Total {} html tags contains in product_title'.format(features.product_title.str.count('<br$').values.sum()))
print('Total {} html tags contains in brand'.format(features.brand.str.count('<br$').values.sum()))
print('Total {} html tags contains in search_term'.format(features.search_term.str.count('<br$').values.sum()))

In [ ]:
from bs4 import BeautifulSoup
def remove_tags(raw_html):
    return BeautifulSoup(raw_html, "lxml").text

In [ ]:
features['product_description'] = features['product_description'].apply(remove_tags)
features['combined_attr'] = features['combined_attr'].apply(remove_tags)

In [ ]:
print('Total {} html tags contains in product description'.format(features.product_description.str.count('<br$').values.sum()))
print('Total {} html tags contains in combined_attr'.format(features.combined_attr.str.count('<br$').values.sum()))
print('Total {} html tags contains in product_title'.format(features.product_title.str.count('<br$').values.sum()))
print('Total {} html tags contains in brand'.format(features.brand.str.count('<br$').values.sum()))
print('Total {} html tags contains in search_term'.format(features.search_term.str.count('<br$').values.sum()))

5. Stopwords removal by spacy nlp small model

In [ ]:
stopwords = nlp.Defaults.stop_words
print(len(stopwords))
stop = set(stopwords)
#stopwords.remove('not')
#stopwords.update('not only')

6. Lemmetisation using pattern library

In [ ]:
#!conda install -c asmeurer pattern -y
import pattern
from pattern.en import lemma

In [ ]:
def clean(x):
    result = x.lower()
    result = result.split()
    result = " ".join([lemma(word) for word in result
                     if word not in stop])
    return result

In [ ]:
features['product_title'] = features['product_title'].apply(clean)
features['search_term'] = features['search_term'].apply(clean)
features['brand'] = features['brand'].apply(clean)
features['product_description'] = features['product_description'].apply(clean)
features['combined_attr'] = features['combined_attr'].apply(clean)

FEATURE ENGINEERING (part of text mining, NLP)

Import the libraries needed for feature engineering

In [ ]:
import pickle
import keras as kr
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
from keras.layers import *
import spacy
import re
import math 
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder



Find null values in features final dataset just to ensure that null values in only relevance column

In [ ]:
for col in features.columns:
  print('{} - {} null values'.format(col, features[col].isna().values.sum()))

Install textdistance to find the distance measures of jaccard, levenshtein distances

In [ ]:
pip install textdistance

Find the numerical values of jaccard, levenshtein distances for all the independent variable columns in features dataset
JACCARD - a measure of dissimilarity
LEVENSHTEIN - the minimum number of single-character edits required to change one word into the other.

In [ ]:
import textdistance
#with description
features["jaccard_similar_desc"] = [textdistance.jaccard(features["search_term"][i], features["product_description"][i]) for i in range(0, len(features))]
features["levenshtein_similarz_desc"] = [textdistance.levenshtein(features["search_term"][i], features["product_description"][i]) for i in range(0, len(features))]
#with title
features["jaccard_similar_title"] = [textdistance.jaccard(features["search_term"][i], features["product_title"][i]) for i in range(0, len(features))]
features["levenshtein_similarz_title"] = [textdistance.levenshtein(features["search_term"][i], features["product_title"][i]) for i in range(0, len(features))]
#with brand
features["jaccard_similar_brand"] = [textdistance.jaccard(features["search_term"][i], features["brand"][i]) for i in range(0, len(features))]
features["levenshtein_similarz_brand"] = [textdistance.levenshtein(features["search_term"][i], features["brand"][i]) for i in range(0, len(features))]
#with combined attr
features["jaccard_similar_CA"] = [textdistance.jaccard(features["search_term"][i], features["combined_attr"][i]) for i in range(0, len(features))]
features["levenshtein_similarz_CA"] = [textdistance.levenshtein(features["search_term"][i], features["combined_attr"][i]) for i in range(0, len(features))]

In [ ]:
features.head()

In [ ]:
features.to_csv(r'/kaggle/working/features.csv', index=False)

Text embedding process - with embed similarity between the search term and other IV columns to find items similar to a given item. 

In [ ]:
import xgboost
import gensim
from time import time
from gensim.models import KeyedVectors
from gensim.utils import simple_preprocess, tokenize
from nltk.corpus import brown
embed_model = gensim.models.Word2Vec(brown.sents())
embed_model.save('brown.embedding')
model = gensim.models.Word2Vec.load('brown.embedding')

In [ ]:
%%time
def embeding_similarity_calculator(s, t, i):
    _sum = 0
    avg = 0
    if len(s.split()) == 0 :
        return 0
    for s_word in s.split():
        _max = 0
        for t_word in t.split():
            if ((s_word in model.wv) and (t_word in model.wv)):
                _max = max(_max, model.wv.similarity(s_word, t_word))
        _sum += _max
    avg = _sum/ len(s.split())
    return avg
features["simdesc"] = [embeding_similarity_calculator(features["search_term"][i], features["product_description"][i],i) for i in range(0, len(features))]
features["simtitle"] = [embeding_similarity_calculator(features["search_term"][i], features["product_title"][i],i) for i in range(0, len(features))]
features["simbrand"] = [embeding_similarity_calculator(features["search_term"][i], features["brand"][i],i) for i in range(0, len(features))]
features["simcomb"] = [embeding_similarity_calculator(features["search_term"][i], features["combined_attr"][i],i) for i in range(0, len(features))]
features.head()

Import the library for TF-IDF step for cosine similarity - a metric that measures the cosine of the angle between two vectors projected in a multi-dimensional space.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidf_vect = TfidfVectorizer(analyzer='char_wb', ngram_range = (3,3), max_features = 1500)
tfidf_des = tfidf_vect.fit_transform(features.product_description).toarray()
tfidf_brand = tfidf_vect.fit_transform(features.brand).toarray()
tfidf_CA = tfidf_vect.fit_transform(features.combined_attr).toarray()
tfidf_title = tfidf_vect.fit_transform(features.product_title).toarray()
tfidf_search = tfidf_vect.transform(features.search_term).toarray()

In [ ]:
import spacy
from scipy.spatial import distance
features["tfidfCS_desc"] = [distance.cosine(tfidf_search[i], tfidf_des[i]) for i in range(0, len(tfidf_des))]
features["tfidfCS_title"] = [distance.cosine(tfidf_search[i], tfidf_title[i]) for i in range(0, len(tfidf_title))]
features["tfidfCS_brand"] = [distance.cosine(tfidf_search[i], tfidf_brand[i]) for i in range(0, len(tfidf_brand))]
features["tfidfCS_CA"] = [distance.cosine(tfidf_search[i], tfidf_CA[i]) for i in range(0, len(tfidf_CA))]
                                                                                                


MODEL Selection

Drop the categorical columns from the datasets and have only numerical columns for regression

In [ ]:
import pandas as pd
features = pd.DataFrame(features).fillna(0)
mod_train = features.iloc[:74067]
mod_test = features.iloc[74067:]
y_train = mod_train['relevance']
mod_train = mod_train.drop(columns=['product_title','product_description','brand','search_term','relevance','combined_attr'])
mod_test = mod_test.drop(columns=['product_title','product_description','brand','search_term', 'relevance','combined_attr'])

In [ ]:
print(len(mod_train), ' ',len(mod_test))

METRIC


Define a function to evaluate the model based on MAE, MSE metrics

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder

def modelEvaluate(model, X_train, y_train, x_val, y_val, label_encode = None):
    model.fit(X_train, y_train)
    pred = model.predict(x_val)
    _pred = pred
    _y_val = y_val
    if label_encode is not None:
        _y_val = label_encode.inverse_transform(_y_val)
        _pred = label_encode.inverse_transform(_pred)
    mae, mse = (mean_absolute_error(_pred, _y_val),mean_squared_error(_pred, _y_val))
    return (mae, mse)

In [ ]:
from sklearn.model_selection import KFold
import math 
def crossValidate(model, label_encoder = None):
    mae_score = []
    mse_score = []
    kf = KFold(n_splits=4)
    kf.get_n_splits(mod_train)
    for train_index, test_index in kf.split(mod_train):
        X, X_test = mod_train.iloc[train_index], mod_train.iloc[test_index]
        y, y_test = y_train.iloc[train_index], y_train.iloc[test_index]
        (mae, mse) = modelEvaluate(model, X, y, X_test, y_test, label_encode = label_encoder)
        mae_score.append(mae)
        mse_score.append(mse)
    return [sum(mae_score)/len(mae_score), math.sqrt(sum(mse_score)/len(mse_score))]

REGRESSION

Cross validation model to compare models of Random forest, XG Boost and Gradient Boosting with MAE, MSE scores

In [ ]:
from sklearn import preprocessing
from sklearn.svm import SVR
from sklearn.ensemble import ExtraTreesRegressor, GradientBoostingRegressor, RandomForestRegressor
from sklearn import linear_model
import xgboost
from xgboost import XGBRegressor
result = []
result.append(('RandomForest',crossValidate(RandomForestRegressor(n_estimators=700, max_depth=6, random_state=42))))
result.append(('XGBoost',crossValidate(XGBRegressor(colsample_bytree=0.4,       
                 learning_rate=0.1,
                 max_depth=6,
                 n_estimators=700,                                                                    
                 reg_alpha=0.075,
                 reg_lambda=0.045,
                 subsample=0.55,
                 seed=42))))
result.append(('GradientBoosting',crossValidate(GradientBoostingRegressor(n_estimators=700, max_depth=6, random_state=42))))

In [ ]:
result

Model Building of Stacked regressor combining the estimators of random forest, xgboost and taking final estimator from BayesianRidge

In [ ]:
%%time
from sklearn.ensemble import ExtraTreesRegressor, GradientBoostingRegressor, RandomForestRegressor
from xgboost import XGBRegressor
from sklearn import linear_model
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import BayesianRidge
estimators = [  ('RandomForest', RandomForestRegressor(n_estimators=700, max_depth=6, random_state=42)),
                ('XGBoost', XGBRegressor(colsample_bytree=0.4,learning_rate=0.1,max_depth=6,n_estimators=700,reg_alpha=0.075,reg_lambda=0.045,
                                         subsample=0.55,seed=42)),                                                                    
               
             ]
stacked_reg = StackingRegressor(
        estimators=estimators,
        final_estimator=BayesianRidge()
    )
result.append(('StackedBayesianRidge',crossValidate(stacked_reg)))

Plot the MAE,RMSE of the 4 models - RF, XGBoost, Gradient, StackedBayesian

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
labels = []
mae = []
rmse = []
for _name, _score in result:
    labels.append(_name)
    mae.append(_score[0])
    rmse.append(_score[1])

#Plotting
x = np.arange(len(labels))  # the label locations
width = 0.3  # the width of the bars

fig, ax = plt.subplots(figsize=(15,6))
rects1 = ax.bar(x - width/2, mae, width, label='MAE')
rects2 = ax.bar(x + width/2, rmse, width, label='RMSE')

ax.set_ylabel('Scores')
ax.set_title('Scores by models')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()

ax.bar_label(rects1, padding=3)
ax.bar_label(rects2, padding=3)

fig.tight_layout()

plt.show()

Fitting the model on train dataset and validating the model for prediction of relevance scores in test dataset

In [ ]:
stacked_reg.fit(mod_train, y_train)
pred = stacked_reg.predict(mod_test)
pred.shape

Determine the importance of features in the model and arranging them in the order of highest importance

In [ ]:
fi = pd.DataFrame({'importance': stacked_reg.feature_names_in_}).sort_values(by='importance', ascending=False)
#fi = fi.reset_index()
fi.head(20)

Generate the submission file with the relevance scores that are predicted from the stacked regressor model

In [ ]:
submission = pd.read_csv('/kaggle/input/sample/sample_submission.csv')
submission['relevance'] = pred
submission.to_csv('submission.csv', index=False)